<a href="https://colab.research.google.com/github/mastertpf/3d/blob/main/Interpolation_Lineaer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lineær interpolation — klassisk opsætning + eksport til Word

Enkel, traditionel opsætning: **kun lineær interpolation** (rette linjer mellem datapunkter).
- Separate felter til **x** og **y** (kommaseparerede lister)
- Indtast et **x** (forespørgsel) → få **y**
- Plot af kurven og beregningen
- **Eksportér** til Word (.docx) med plot og detaljer

Data sorteres efter stigende x. Ligger x udenfor intervallet, kan du slå ekstrapolation til.

In [55]:
!pip -q install ipywidgets python-docx
from IPython.display import display, Markdown, clear_output
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets # Corrected import statement
from ipywidgets import VBox, HBox, Text, FloatText, Checkbox, Button, Output, Layout, HTML
from math import isfinite
from docx import Document
from docx.shared import Inches
import os

In [56]:
# Hjælpefunktioner (ingen docstrings for at undgå citattegn-konflikter)
def parse_list(txt):
    if not txt.strip():
        return []
    txt = txt.replace(';', ',')
    parts = [p.strip() for p in txt.split(',') if p.strip()]
    vals = []
    for p in parts:
        try:
            v = float(p)
            if not isfinite(v):
                raise ValueError
            vals.append(v)
        except:
            raise ValueError(f"Kan ikke tolke tallet: '{p}'")
    return vals

def interval_for_x(xs, x):
    i = np.searchsorted(xs, x) - 1
    if i < 0 or i >= len(xs) - 1:
        return None
    return i

def linseg_y(x1, y1, x2, y2, x):
    if x1 == x2:
        raise ValueError('x1 og x2 må ikke være ens.')
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1
    return a * x + b, a, b

def piecewise_linear(xs, ys, Xq, allow_extrap=False):
    xs = np.asarray(xs, dtype=float)
    ys = np.asarray(ys, dtype=float)
    order = np.argsort(xs)
    xs, ys = xs[order], ys[order]

    Yq = np.full_like(Xq, np.nan, dtype=float)
    seg_info = [None]*len(Xq)

    for k, xv in enumerate(Xq):
        i = interval_for_x(xs, xv)
        if i is None:
            if not allow_extrap:
                continue
            if xv < xs[0]:
                i = 0
            else:
                i = len(xs) - 2
        yv, a, b = linseg_y(xs[i], ys[i], xs[i+1], ys[i+1], xv)
        Yq[k] = yv
        seg_info[k] = (i, a, b)
    return xs, ys, Yq, seg_info


In [57]:
# Widgets — lineær interpolation + eksport til Word
x1_widget = Text(value='1,0', description='x1:', layout=Layout(width='200px'))
y1_widget = Text(value='2,0', description='y1:', layout=Layout(width='200px'))
x2_widget = Text(value='3,0', description='x2:', layout=Layout(width='200px'))
y2_widget = Text(value='8,0', description='y2:', layout=Layout(width='200px'))
x_query = Text(value='1,5', description='x (forespørgsel):', layout=Layout(width='200px'))

xlabel_widget = Text(value='x', description='x-akse titel (max 32):', layout=Layout(width='300px'))
ylabel_widget = Text(value='y', description='y-akse titel (max 32):', layout=Layout(width='300px'))

axis_options = [
    ('Brugerdefineret', ('x', 'y')),
    ('Kt', ('Temperatur (°C)', 'Korrektion Kt')),
    ('Ktm', ('Varmeledning (K*m/W)', 'Ktm')),
    ('Ks', ('Antal (N)','Korrektion Ks'))
]
axis_dropdown = ipywidgets.Dropdown(
    options=axis_options,
    value=('x', 'y'),
    description='Akse-sæt:',
    disabled=False,
)

def update_axis_labels(change):
    xlabel_widget.value = change['new'][0]
    ylabel_widget.value = change['new'][1]

axis_dropdown.observe(update_axis_labels, names='value')

# Reference Dropdown
reference_options = [
    ('Ingen', ''),
    ('DS-HD-60346-5_2011 + Tabel-', 'DS-HD-60364-5_2011 + Tabel-'),
    ('DS-IEC-60502-2_2014 + Tabel- ', 'DS-IEC-60502-2_2014 + Tabel-'),
]
reference_dropdown = ipywidgets.Dropdown(
    options=reference_options,
    value='',
    description='Tabel reference:',
    disabled=False,
    layout=Layout(width='300px')
)

reference_custom_widget = Text(value='', description='Tilføj til reference:', layout=Layout(width='300px'))


allow_extrap = Checkbox(value=False, description='Tillad ekstrapolation')

run_btn = Button(description='Beregn og tegn', button_style='primary')
export_btn = Button(description='Eksportér til Word (.docx)')
out = Output()
plot_path = 'interpolation_plot.png'
docx_path = 'Lineaer_interpolation_rapport.docx'

_state = {'xs': None, 'ys': None, 'xq': None, 'yq': None, 'a': None, 'b': None, 'i': None, 'xlabel': 'x', 'ylabel': 'y', 'reference': None}

def parse_float_input(txt):
    txt = txt.strip().replace(',', '.')
    try:
        v = float(txt)
        if not isfinite(v):
            raise ValueError
        return v
    except:
        raise ValueError(f"Kan ikke tolke tallet: '{txt.replace('.', ',')}'")


def on_run(_):
    with out:
        clear_output()
        try:
            x1 = parse_float_input(x1_widget.value)
            y1 = parse_float_input(y1_widget.value)
            x2 = parse_float_input(x2_widget.value)
            y2 = parse_float_input(y2_widget.value)
            xq = parse_float_input(x_query.value)

            xlabel = xlabel_widget.value[:32]
            ylabel = ylabel_widget.value[:32]

            reference_dropdown_value = reference_dropdown.value
            reference_custom_value = reference_custom_widget.value.strip()

            if reference_dropdown_value and reference_custom_value:
                 reference = f"{reference_dropdown_value}{reference_custom_value}"
            elif reference_dropdown_value:
                 reference = reference_dropdown_value
            elif reference_custom_value:
                 reference = reference_custom_value
            else:
                 reference = ""


            if x1 == x2:
                 raise ValueError('x1 and x2 must be different.')

            xs = np.array([x1, x2])
            ys = np.array([y1, y2])

            # Ensure points are sorted by x
            order = np.argsort(xs)
            xs_s, ys_s = xs[order], ys[order]

            # Find the interval for the query point
            i = interval_for_x(xs_s, xq)

            # Check if the query point is outside the interval and extrapolation is allowed
            if i is None:
                if not allow_extrap.value:
                     raise ValueError('x is outside the range of data points. Extrapolation is not allowed.')
                # If extrapolation is allowed, use the first or last interval
                if xq < xs_s[0]:
                    i = 0
                else:
                    i = len(xs_s) - 2


            yq, a, b = linseg_y(xs_s[i], ys_s[i], xs_s[i+1], ys_s[i+1], xq)

            display(Markdown(
                f"""**Resultat (lineær):** For **{xlabel} = {xq:g}** fås **{ylabel} = {yq:g}**

Brugte interval: [{xs_s[i]:g}, {xs_s[i+1]:g}] → [{ys_s[i]:g}, {ys_s[i+1]:g}]  \n
Ligning: \\({ylabel} = a {xlabel} + b\\) med \\(a = {a:g}\\), \\(b = {b:g}\\)."""
            ))

            # Plotting
            xmin, xmax = float(np.min(xs_s)), float(np.max(xs_s))
            span = xmax - xmin if xmax > xmin else 1.0
            xlo = xmin - 0.1*span
            xhi = xmax + 0.1*span
            X = np.linspace(xlo, xhi, 400)

            # Simplified for linear interpolation between two points
            Y = a * X + b

            plt.figure(figsize=(7, 4.5))
            plt.plot(X, Y, label='Lineær interpolation')
            plt.scatter(xs_s, ys_s, marker='o', label='Kendte punkter')
            plt.scatter([xq], [yq], marker='x', s=80, label=f'Interpoleret ({xq:g},{yq:g})')
            plt.xlabel(xlabel); plt.ylabel(ylabel)
            plt.title('Lineær interpolation')
            plt.grid(True); plt.legend(); plt.tight_layout()

            # Add reference to the plot
            if reference:
                 plt.figtext(0.99, 0.01, f'Reference: {reference}', horizontalalignment='right', fontsize=9, color='gray')


            plt.savefig(plot_path, dpi=200)
            plt.show()

            _state.update(dict(xs=xs_s, ys=ys_s, xq=float(xq), yq=float(yq), a=float(a), b=float(b), i=int(i), xlabel=xlabel, ylabel=ylabel, reference=reference))
            display(Markdown(f'Plot gemt som `{plot_path}` (bruges til eksport).'))

        except Exception as e:
            display(Markdown(f'**Fejl:** {e}'))

def on_export(_):
    with out:
        try:
            if _state['xs'] is None:
                raise RuntimeError("Kør først 'Beregn og tegn', så der er noget at eksportere.")

            doc = Document()
            doc.add_heading('Lineær interpolation — rapport', level=1)

            xs = _state['xs']; ys = _state['ys']
            xq = _state['xq']; yq = _state['yq']
            a = _state['a']; b = _state['b']; i = _state['i']
            xlabel = _state['xlabel']; ylabel = _state['ylabel']
            reference = _state['reference']

            if reference:
                doc.add_paragraph(f'Reference: {reference}')

            p = doc.add_paragraph(); p.add_run('Inddata (klassisk tabel):').bold = True
            for xv, yv in zip(xs, ys):
                doc.add_paragraph(f'{xlabel} = {xv:g}\t {ylabel} = {yv:g}')

            doc.add_paragraph('')
            doc.add_paragraph(f'Forespørgsel: {xlabel} = {xq:g}')
            doc.add_paragraph(f'Valgt interval: [{xs[i]:g}, {xs[i+1]:g}] → [{ys[i]:g}, {ys[i+1]:g}]')
            doc.add_paragraph(f'Ligning: {ylabel} = a {xlabel} + b med a = {a:g}, b = {b:g}')
            doc.add_paragraph(f'Interpoleret værdi: {ylabel}({xlabel}={xq:g}) = {yq:g}')

            doc.add_paragraph('')
            doc.add_paragraph('Beregning:')
            doc.add_paragraph(f'Formel for lineær interpolation: {ylabel} = y₁ + ({xlabel} - x₁) * (y₂ - y₁) / (x₂ - x₁)')
            doc.add_paragraph(f'Med indsatte værdier: {ylabel} = {ys[i]:g} + ({xq:g} - {xs[i]:g}) * ({ys[i+1]:g} - {ys[i]:g}) / ({xs[i+1]:g} - {xs[i]:g})')
            doc.add_paragraph(f'Resultat: {ylabel} = {yq:g}')

            if os.path.exists(plot_path):
                doc.add_paragraph('')
                doc.add_paragraph('Kurve og datapunkter:')
                doc.add_picture(plot_path, width=Inches(5.8))

            doc.save(docx_path)
            display(Markdown(f'**Eksporteret:** `{docx_path}` er oprettet.'))
        except Exception as e:
            display(Markdown(f'**Fejl ved eksport:** {e}'))

run_btn.on_click(on_run)
export_btn.on_click(on_export)

ui = VBox([
    HTML('<h3>Indtast data</h3>'),
    HBox([x1_widget, y1_widget]),
    HBox([x2_widget, y2_widget]),
    HBox([x_query, allow_extrap]),
    axis_dropdown,
    HBox([xlabel_widget, ylabel_widget]),
    HBox([reference_dropdown, reference_custom_widget]), # Place dropdown and custom text side-by-side
    HBox([run_btn, export_btn]),
    out
])

display(ui)
# Kør én gang for at vise UI